In [4]:
!pip install langchain langchain-community ollama

from langchain_community.llms import Ollama

llm = Ollama(model="deepseek-r1")

response = llm.invoke("What is LangChain?")
print(response)

<think>

</think>

LangChain, short for Language Chain, is a framework developed by OpenAI that facilitates the creation of complex reasoning models using language models. It allows users to build applications that can generate, analyze, and reason about text in a structured way.

### Key Components:
1. **Base Model**: A pre-trained language model (like GPT) that handles tokenization, attention, and generates text.
2. **Chain of Thought**: This feature enables the system to simulate human-like reasoning by chaining together previous responses and new inputs to draw conclusions.
3. **Explainable AI (XAI)**: LangChain integrates with tools like `explanation-building` to make AI decisions more transparent.

### Use Cases:
- **Problem Solving**: Helps in solving complex problems by breaking them down into smaller steps.
- **Creative Writing**: Assists writers by generating prompts or ideas and refining content.
- **Education**: Provides educational tools for learning and teaching, such as 